<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training_vgg16_bn_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%time
!rm -r /content/cloned-repo
%cd ..
!git clone -l -s git://github.com/kavyajeetbora/EAST.git /content/cloned-repo
%cd /content/cloned-repo
!ls

/
Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 157 (delta 42), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (157/157), 6.44 MiB | 9.84 MiB/s, done.
Resolving deltas: 100% (87/87), done.
/content/cloned-repo
dataset.py		    EAST_resnet.py		  model.py
detect.py		    EAST_training.ipynb		  model_resnet.py
dsiplay_ground_truth.ipynb  EAST_training_vgg16_bn.ipynb  train.py
EAST_detection.ipynb	    loss.py
CPU times: user 71.8 ms, sys: 29.8 ms, total: 102 ms
Wall time: 7.53 s


In [0]:
from dataset import *

In [5]:
# AI4Bharat dataset
image_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Images/Images.Zip (Unzipped Files)'
label_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotations/Annotations.Zip (Unzipped Files)'

image_files = sorted(os.listdir(image_folder))
annotation_files = sorted(os.listdir(label_folder))
print('Total files in Image folder',len(image_files), '| Total files in Annotation folder',len(annotation_files))

training_dataset = custom_dataset(image_folder, label_folder)
print(len(training_dataset))

Total files in Image folder 5000 | Total files in Annotation folder 5000
5000


## Dataset Initial Check

In [6]:
index = np.random.randint(low=1,high=len(image_files)+1)
img = Image.open(os.path.join(image_folder,image_files[index]))
label = os.path.join(label_folder,annotation_files[index])
with open(label, 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[-164.0396114   378.22783562 -101.06498158  387.23099059 -104.3498229
  418.93126253 -168.5355033   409.75496996]
--------------------
[-205.73594208  349.99526696 -164.38730366  354.66936312 -168.71027664
  384.98391537 -208.67494557  379.27037472]
--------------------
[-206.42390892  406.81286868 -163.86421993  411.66010244 -167.84135507
  439.54949051 -210.22812515  433.48967466]
--------------------
[293.21133099 343.37704632 356.87578208 321.54623626 366.73896
 347.70239161 303.07450892 369.53320168]
--------------------
[1 1 1 1]


In [7]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(56.)

## Loss for one batch

In [8]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=12)
imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))
print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())
print(torch.sum(gt_score))

torch.Size([12, 3, 512, 512]) torch.Size([12, 1, 128, 128]) torch.Size([12, 5, 128, 128]) torch.Size([12, 1, 128, 128])
tensor(5489.)
CPU times: user 4.72 s, sys: 6.16 s, total: 10.9 s
Wall time: 13 s


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
%%time
from model import EAST
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)
imgs, gt_score, gt_geo, ignored_maps = imgs.to(device), gt_score.to(device), gt_geo.to(device), ignored_maps.to(device)
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([12, 1, 128, 128]) torch.Size([12, 5, 128, 128])
CPU times: user 4.16 s, sys: 935 ms, total: 5.09 s
Wall time: 5.07 s


In [11]:
from loss import Loss

loss_fn = Loss()
loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.90340304, angle loss is 0.07636017, iou loss is 0.78137517


tensor(2.4484, device='cuda:0', grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    #scheduler = lr_scheduler.StepLR(optimizer, step_size=epoch_iter, gamma=0.1)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      #scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, 'model_epoch_{}.pth'.format(epoch+1))
      
      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))
    return loss_curve

In [14]:
%%time
loss_curve = train(model, image_folder, label_folder, batch_size=4, lr=1e-4, epoch_iter=10, interval = 1)

5000


RuntimeError: ignored

In [0]:
plt.plot(loss_curve)

[Adam and AdamW algorithms](https://www.fast.ai/2018/07/02/adam-weight-decay/)